In [1]:
from korean_crawling_bundles import *

In [2]:
word_df = pd.read_csv('konlpy_0917_link.csv')
word_df.head()

,corpus
0,"['미', '미중', '중', '무역', '무역분쟁', '분쟁', '격화', '아이..."
1,"['5', '먹거리', '통', '통사', '사', '게임', '스포츠', '눈',..."
2,"['재생', '재생에너지', '에너지', '확대', '찬성', '86', '발전소'..."
3,"['금리', '인하', '고민', '상황', '연', '연내', '내', '통화',..."
4,"['무역', '무역협상', '협상', '재개', '제안', '거부', '분위기', ..."


In [3]:
for i in range(len(word_df)) : 
    word_df.iat[i,0] = word_df.iat[i,0][1:-1]
    
word_ls = []
for i in range(len(word_df)) : 
    new_ls = word_df.iat[i,0].split(',')

    for i in range(len(new_ls)) : 
        new_ls[i] = new_ls[i].replace(' ','')
    
    for i in range(len(new_ls)) : 
        new_ls[i] = new_ls[i].replace("'",'')
    
    word_ls += new_ls
print(len(word_ls))

for i in range(len(word_df)):
    word_df.iat[i,0] = ''.join(word_df.iat[i,0]).replace("'","").split(',')
    
for i in range(len(word_df)):
    word_df.iat[i,0] = ','.join(word_df.iat[i,0]).replace(' ','').split(',')

25708


In [4]:
word_df.tail()

,corpus
88,"[데일리, 데일리안, 안, 이정, 이정윤, 윤, 기자, 서울시, 영등포, 영등포구,..."
89,"[데일리, 데일리안, 안, 이정, 이정윤, 윤, 기자, 동네, 쪽, 쪽방촌, 방촌,..."
90,"[민, 민관협력, 관, 협력, 노후, 노후주택, 주택, 개선, 개선사업, 사업, 개..."
91,"[추석, 활기, 광장, 광장시장, 시장, 사진, 뉴, 뉴시스, 시스, 식품, 외식,..."
92,"[앵커, 남북, 경제, 경제협력, 협력, 시작, 30, 30년, 년, 전, 관계, ..."


In [53]:
print(len(word_df.iat[0,0]))
for i in range(len(word_df)):
    for idx,j in enumerate(word_df.iat[i,0]):
        if len(j) : 
            word_df.iat[i,0].pop(idx)
print(len(word_df.iat[0,0]))

1
0


In [12]:
documents = [i for i in word_df['corpus']]
K = 5

In [18]:
from collections import Counter

# 각 토픽이 각 문서에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 문서를 의미
document_topic_counts = [Counter() for _ in documents]

# 각 단어가 각 토픽에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 토픽을 의미
topic_word_counts = [Counter() for _ in range(K)]

# 각 토픽에 할당되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 토픽을 의미함
topic_counts = [0 for _ in range(K)]

# 각 문서에 포함되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 문서를 의미함
document_lengths = [len(doc) for doc in documents]

# 단어 종류의 수
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)

# 총 문서의 수
D = len(documents)

In [19]:
def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

In [20]:
def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

import random
def sample_from(weights):
    # i를 weights[i] / sum(weights)
    # 확률로 반환
    total = sum(weights)
    # 0과 total 사이를 균일하게 선택
    rnd = total * random.random()
    # 아래 식을 만족하는 가장 작은 i를 반환
    # weights[0] + ... + weights[i] >= rnd
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

In [21]:
random.seed(0)

# topic 수 지정

# 각 단어를 임의의 토픽에 랜덤 배정
document_topics = [[random.randrange(K) for word in document]
                    for document in documents]

# 위와 같이 랜덤 초기화한 상태에서 
# AB를 구하는 데 필요한 숫자를 세어봄
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [22]:
for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            # 깁스 샘플링 수행을 위해
            # 샘플링 대상 word와 topic을 제외하고 세어봄
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # 깁스 샘플링 대상 word와 topic을 제외한 
            # 말뭉치 모든 word의 topic 정보를 토대로
            # 샘플링 대상 word의 새로운 topic을 선택
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # 샘플링 대상 word의 새로운 topic을 반영해 
            # 말뭉치 정보 업데이트
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

In [43]:
document_topic_counts[0]

Counter({0: 0, 1: 146, 2: 0, 3: 0, 4: 0})

In [40]:
document_topic_counts[1]

Counter({0: 39, 1: 0, 2: 212, 3: 0, 4: 0})